In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.circuit.library import QFT
from math import gcd
# Define the quantum register
n = 5
qr = QuantumRegister(n+n)

# Define a classical register to measure the results
cr = ClassicalRegister(n)

# Initialize the quantum circuit
circuit = QuantumCircuit(qr, cr)

# Define values for a and N, N must be the sum of 2 prime numbers
a = 8
N = 21


In [2]:
# Initialise counting qubits in state |+>
circuit.h(range(5))
# And auxiliary register in state |1>
circuit.x(n+n-1)

In [3]:
# a^x mod 21
def modExpon(a, x):
    # a is chosen so its coprime with N and
    # a^r mod N = 1 (We want trivial period(1) as that is a key step in alg for it being faster than classical)
    if a not in [2,5,8,10,11,13,19]:
        raise ValueError("a must be 2,5,8,10,11,13,19")
        
    #need 5 qubits to represent N(21)    
    qc = QuantumCircuit(n)
    
    #implementing the swap gates for rotation
    for i in range(x):
        if a in [8]:
            qc.x(1)
            qc.x(3)
        if a in [13]:
            qc.x(2)
            qc.x(3)
        if a in [2,5,10,11,19]:
            qc.swap(1,2)
            qc.swap(2,3)
            qc.swap(3,4)
            qc.swap(1,4)
        
                
    qc = qc.to_gate()
    qc.name = "%i^%i mod %i" % (a, x, N)
    c_qc = qc.control()
    
    return c_qc

for x in range(n):
        power = 2**x
        circuit.append(modExpon(a, power), [x] + list(range(n, n+n)))

In [4]:
# Apply the inverse Quantum Fourier Transform (QFT)
circuit.append(QFT(n, do_swaps=False).inverse(), (range(n)))

# Measure qubits
circuit.measure(0,0)
circuit.measure(1,1)
circuit.measure(2,2)
circuit.measure(3,3)
circuit.measure(4,4)


# Simulate the circuit to obtain measurement results
simulator = Aer.get_backend('aer_simulator')
job = execute(circuit, simulator, shots=1000)
result = job.result()

counts = result.get_counts(circuit)
print(counts)
circuit.draw(fold=-1)

{'00101': 9, '11101': 29, '00000': 509, '00001': 198, '11111': 187, '00011': 25, '01011': 4, '10101': 3, '01111': 2, '11001': 10, '01001': 4, '10011': 4, '00111': 6, '01101': 1, '11011': 8, '10111': 1}


┌───┐                                                                            ┌───────┐┌─┐            
q0_0: ┤ H ├───────■────────────────────────────────────────────────────────────────────┤0      ├┤M├────────────
      ├───┤       │                                                                    │       │└╥┘┌─┐         
q0_1: ┤ H ├───────┼──────────────■─────────────────────────────────────────────────────┤1      ├─╫─┤M├─────────
      ├───┤       │              │                                                     │       │ ║ └╥┘┌─┐      
q0_2: ┤ H ├───────┼──────────────┼──────────────■──────────────────────────────────────┤2 IQFT ├─╫──╫─┤M├──────
      ├───┤       │              │              │                                      │       │ ║  ║ └╥┘┌─┐   
q0_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■───────────────────────┤3      ├─╫──╫──╫─┤M├───
      ├───┤       │              │              │              │                       │       │ ║  ║  ║ └╥┘┌─┐
q0_4: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────■────────┤4      ├─╫──╫──╫──╫─┤M├
      └───┘┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴───────┐└───────┘ ║  ║  ║  ║ └╥┘
q0_5: ─────┤0            ├┤0            ├┤0            ├┤0            ├┤0             ├──────────╫──╫──╫──╫──╫─
           │             ││             ││             ││             ││              │          ║  ║  ║  ║  ║ 
q0_6: ─────┤1            ├┤1            ├┤1            ├┤1            ├┤1             ├──────────╫──╫──╫──╫──╫─
           │             ││             ││             ││             ││              │          ║  ║  ║  ║  ║ 
q0_7: ─────┤2 8^1 mod 21 ├┤2 8^2 mod 21 ├┤2 8^4 mod 21 ├┤2 8^8 mod 21 ├┤2 8^16 mod 21 ├──────────╫──╫──╫──╫──╫─
           │             ││             ││             ││             ││              │          ║  ║  ║  ║  ║ 
q0_8: ─────┤3            ├┤3            ├┤3            ├┤3            ├┤3             ├──────────╫──╫──╫──╫──╫─
      ┌───┐│             ││             ││             ││             ││              │          ║  ║  ║  ║  ║ 
q0_9: ┤ X ├┤4            ├┤4            ├┤4            ├┤4            ├┤4             ├──────────╫──╫──╫──╫──╫─
      └───┘└─────────────┘└─────────────┘└─────────────┘└─────────────┘└──────────────┘          ║  ║  ║  ║  ║ 
c0: 5/═══════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩══╩═
                                                                                                 0  1  2  3  4

In [5]:
for i in counts:
    # Reverses order of qubits as in q circuits its ordered opposite way round compared to normal bits, turns binary to denary
    measuredValue = int(i[::-1], 2)

    if measuredValue % 2 != 0:
        # Needs to be even so it can be divided by 2
        print("Value is not even")
        continue
    # a^(p/2) +- 1 = number that shares factors with N
    y = int((a ** (measuredValue/2)) % N)
    if (y + 1) % N == 0:
        continue
    
    factors = gcd(y + 1, N), gcd(y - 1, N)
    print(factors)

(1, 21)
Value is not even
(1, 21)
(1, 21)
Value is not even
(1, 21)
(3, 7)
Value is not even
(3, 7)
Value is not even
(3, 7)
Value is not even
(1, 21)
(3, 7)
Value is not even
Value is not even


In [6]:
uniqueFactors = []  # List to store unique factors

for i in counts:
    measuredValue = int(i[::-1], 2)
    
    if measuredValue % 2 != 0:
        continue
    
    x = int((a ** (measuredValue / 2)) % N)
    
    if (x + 1) % N == 0:
        continue
    
    factor1 = gcd(x + 1, N)
    factor2 = gcd(x - 1, N)
    
    # Add factors to list if they are not in already
    if factor1 not in uniqueFactors:
        uniqueFactors.append(factor1)
    if factor2 not in uniqueFactors:
        uniqueFactors.append(factor2)

uniqueFactors.sort()

print(uniqueFactors)


[1, 3, 7, 21]


In [7]:
if len(uniqueFactors) == 4:
    print("The prime factors for",N,"are",uniqueFactors[1],"and",uniqueFactors[2])
    
elif len(uniqueFactors) == 3:
    secFactor = N/uniqueFactors[1]
    print("The prime factors for",N,"are",uniqueFactors[1],"and",secFactor)
else:
    print("Use another value for a")

The prime factors for 21 are 3 and 7
